In [ ]:
from typing import Any, List, Union, Dict
from pydantic import BaseModel, field_validator, ValidationError, PrivateAttr


class SingleTypeMixin:
    """Mixin to enforce only single float values for all fields."""

    @field_validator("*", mode="before")
    @classmethod
    def enforce_single_type(cls, value):
        """Ensure all fields contain only single floats (no lists)."""

        # print("enforce_single_type", value)

        if isinstance(value, list):
            raise ValueError("Lists are not allowed for this class.")
        if isinstance(value, Dict):  # Check for nested dictionaries containing BaseModel instances
            for key, dict_value in value.items():
                if isinstance(dict_value, BaseModel):  # Recursively validate BaseModel objects
                    for field, field_value in dict_value.model_dump().items():
                        if isinstance(field_value, list):
                            raise ValueError(f"Nested dictionary attribute '{key}.{field}' must not be a list.")
        if isinstance(value, BaseModel):  # Validate Pydantic objects
            for field, field_value in value.model_dump().items():
                if isinstance(field_value, list):
                    raise ValueError(f"Nested attribute '{field}' must not be a list.")
                
        return value





from itertools import product
class MultiTemplate(BaseModel):
    """
    """
    _multi_params: Dict = PrivateAttr(default={})  # Private storage
    _sonata_config: Dict = PrivateAttr(default={})
    
    _single_version_class_name: str = PrivateAttr(default="")
    
    
    @property
    def multi_params(self) -> Dict:
        
        for attr_name, attr_value in self.__dict__.items():

            if isinstance(attr_value, Dict) and all(isinstance(dict_val, MultiTemplate) for dict_key, dict_val in attr_value.items()):
                for dict_key, dict_val in attr_value.items():
                    for key, value in dict_val.__dict__.items():
                        if not isinstance(value, BaseModel) and isinstance(value, list) and len(value) > 1:
                            self._multi_params[f"{attr_name}.{dict_key}.{key}"] = {
                                "coord_param_keys": [attr_name, dict_key, key],
                                "coord_param_values": value
                            }
                            
        return self._multi_params


    def generate_grid_scan_coords(self) -> list:

        all_tuples = []
        for key, value in self.multi_params.items():
            tups = []
            for k, v in zip([value["coord_param_keys"] for i in range(len(value['coord_param_values']))], value['coord_param_values']):
                tups.append((k, v))

            all_tuples.append(tups)

        coords = [coord for coord in product(*all_tuples)]
        return coords
    
    def cast_to_single_instance(self):
        # NOT AN IDEAL WAY OF FINDING THE NEW CLASS (Is there an alternative?)
        class_to_cast_to = globals()[self._single_version_class_name] 
        single_instance = class_to_cast_to.model_validate(self.model_dump())
        return single_instance



import copy
class Campaign(BaseModel):

    template_instance: MultiTemplate = None

    _coord_instances: list = PrivateAttr(default=[])

    @property
    def coord_instances(self) -> list[MultiTemplate]:

        if len(self._coord_instances) > 0: return self._coord_instances

        for coord in self.template_instance.generate_grid_scan_coords():

            coord_template_instance = copy.deepcopy(self.template_instance)

            for param in coord:
                keys = param[0]
                val = param[1]

                current_level = coord_template_instance
                for i, key in enumerate(keys):

                    if isinstance(current_level, MultiTemplate):

                        if i == len(keys) - 1:
                            current_level.__dict__[key] = val
                        else:
                            current_level = current_level.__dict__[key]
                
                    elif isinstance(current_level, Dict):
                        current_level = current_level[key]


            try:
                coord_instance = coord_template_instance.cast_to_single_instance()
                self._coord_instances.append(coord_instance)
                
            except ValidationError as e:
                print("Validation Error:", e)

        return self._coord_instances




    
class Stimulus(MultiTemplate):
    """A nested model containing numeric attributes."""
    nested_param1: Union[float, List[float]]
    nested_param2: Union[float, List[float]]

class SimulationCampaignTemplate(MultiTemplate):
    """Base simulation model that contains a generic nested object."""
    nested: Dict[str, Stimulus]

    _single_version_class_name: str = PrivateAttr(default="Simulation")


class Simulation(SimulationCampaignTemplate, SingleTypeMixin):
    """Only allows single float values and ensures nested attributes follow the same rule."""
    pass



nested_multi = Stimulus(nested_param1=[1.5, 2.5], nested_param2=[3.5, 7.0])
simulation_campaign_template = SimulationCampaignTemplate(nested={"config": nested_multi})
campaign = Campaign(template_instance=simulation_campaign_template)
print(campaign.coord_instances)

# print(simulation_campaign_template.multi_params)
# print(simulation_campaign_template.generate_grid_scan_coords())












[Simulation(nested={'config': Stimulus(nested_param1=1.5, nested_param2=3.5)}), Simulation(nested={'config': Stimulus(nested_param1=1.5, nested_param2=7.0)}), Simulation(nested={'config': Stimulus(nested_param1=2.5, nested_param2=3.5)}), Simulation(nested={'config': Stimulus(nested_param1=2.5, nested_param2=7.0)})]


In [ ]:
# # Valid Simulation instance (SingleType) with direct nested object
# nested_single = Stimulus(nested_param1=1.5, nested_param2=2.5)  # ✅ Must be single floats
# print(nested_single)
# simulation = Simulation(param1=2.0, param2=3.0, nested={"config": nested_single})
# print(simulation)

# Valid SimulationCampaignTemplate instance (MultiType) with direct nested object

# # Valid Simulation with nested dictionary containing BaseModel
# simulation_with_dict = Simulation(
#     param1=2.0, 
#     param2=3.0, 
#     nested={"config": Stimulus(nested_param1=2.2, nested_param2=4.4)}
# )
# print(simulation_with_dict)

# # Valid SimulationCampaignTemplate with nested dictionary containing BaseModel
# simulation_campaign_template_with_dict = SimulationCampaignTemplate(
#     param1=[2.0, 4.0], 
#     param2=3.0, 
#     nested={"config": Stimulus(nested_param1=[1.1, 2.2], nested_param2=3.3)}
# )
# print(simulation_campaign_template_with_dict)

# print("\nNew Test\n")



# ❌ Nested dictionary contains a BaseModel with a list (not allowed in Simulation)
try:
    invalid_simulation_dict = Simulation(
        param1=2.0, 
        param2=3.0, 
        nested={"config": Stimulus(nested_param1=[1.5, 2.5], nested_param2=3.5)}
    )
except ValueError as e:
    print(e)

# ❌ Nested dictionary contains a BaseModel with a string (not allowed in SimulationCampaignTemplate)
try:
    invalid_simulation_campaign_template_dict = SimulationCampaignTemplate(
        param1=[2.0, 4.0], 
        param2=3.0, 
        nested={"config": Stimulus(nested_param1="invalid", nested_param2=3.5)}
    )
except ValueError as e:
    print(e)